# Anomaly detection in time series data

Time series is a sequence of values recorded at *equidistant* time points. Broadly, time can be replaced with pixels, energy(wavelength), temperature, etc.


In this tutorial you will learn:

* how to explore and compare univariate time series
* how to transform data, making it suitable for a model, based on your expert knowledge
* unsupervised approach to anomaly detection (elliptical envelope)
* supervised binary classification (logistic regression)
* how to interpret the models
* different metrics for model performance

In [ ]:
import numpy
import pandas
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score

## Generate examples

In [ ]:
from utils import generate_steady_series, generate_anomaly_series

numpy.random.seed(1)

steady_series = generate_steady_series(N_samples=500, max_size = 1000)
anomaly_series = generate_anomaly_series(N_samples=500, max_size = 1000)

## Visually explore the data

Plot a steady signal

In [ ]:
n = 0
fig, ax = plt.subplots(1,2, dpi = 150, figsize = (10, 4))
ax[0].plot(steady_series[n].signal)
ax[0].set_ylim(0, numpy.max(steady_series[n].signal)*1.05)
ax[0].set_xlabel('time')
ax[0].set_ylabel('signal')

ax[1].plot(steady_series[n].signal[1:]-steady_series[n].signal[:-1])
ax[1].set_xlabel('time')
ax[1].set_ylabel('signal`s first derivative')

Plot an anomalous signal

In [ ]:
n = 3
fig, ax = plt.subplots(1,2, dpi = 150, figsize = (10, 4))
ax[0].plot(anomaly_series[n].signal)
ax[0].set_ylim(0, numpy.max(anomaly_series[n].signal)*1.05)
ax[0].set_xlabel('time')
ax[0].set_ylabel('signal')

ax[1].plot(anomaly_series[n].signal[1:]-anomaly_series[n].signal[:-1])
ax[1].set_xlabel('time')
ax[1].set_ylabel('signal`s first derivative')

Convert the generated examples to a pandas.DataFrame

In [ ]:
steady_df = pandas.DataFrame(steady_series)
steady_df.head()

What are the typical length of the `steady` series?

More about Lambda functions: https://www.w3schools.com/python/python_lambda.asp

In [ ]:
lengths_steady = steady_df['signal'].apply(lambda x: len(x))
plt.figure(dpi = 100)
lengths_steady.hist(bins = 20)
plt.xlabel('length of series')
plt.ylabel('counts')

In [ ]:
# # YOUR CODE HERE
# anomaly_df = ...

# ANSWER
anomaly_df = pandas.DataFrame(anomaly_series)

## Generate features: put your expert knowledge into play

### Preprocessing

Think if you need to do any pre-processing on for the data. 

* Is the mean value important? (saturation, too weak signal)
* Is the absolute value of signal variation is important? Can it be that onlt the variation realtive to the mean values are important? (intensity vs peak position)
* Are outlier (extreme) values important or they can be ignored?
* Are there any NaN values? What values they should be replaced with (if replaced)?

Here we chose to center each signal around the mean and normalize by the mean. We ignore 5% of exreme values, such as cosmic rays. 

In [ ]:
from scipy.stats import trim_mean
from scipy.stats.mstats import trimmed_std
from scipy.stats import kurtosis, skew
from statsmodels.tsa.stattools import pacf

from utils import generate_all_features

In [ ]:
def preprocess(series, p_cut = 0.025, nan_replace = 0.0): 
    
    # replace NaN values with zeros
    series = numpy.nan_to_num(series, nan = nan_replace)
    
    #center and normalize by the mean
    series_mean = trim_mean(series, p_cut)
    if series_mean == 0:
        return numpy.array([numpy.nan]*len(series))
    series = series - series_mean
    series = series/series_mean  
    
    return series

How would you modify the preprocessing function if you want to replace NaN values with: 
* the mean of the series
* the median
* any unrealistic (e.g. negative) value ? You need to calculate calculate mean before replacing NaN is this case.

Apply preprocessing to the signals.

In [ ]:
steady_df['scaled_series'] = steady_df['signal'].apply(lambda x: preprocess(x))
anomaly_df['scaled_series'] = anomaly_df['signal'].apply(lambda x: preprocess(x))

### Explore different aggregative statistics

What transformation helps to better separate good and anomalous examples?

Explore different options in the function `feature` 2 cells below. A few options to consider:

* standard deviation, variance
* kurtosis
* skew
* autocorrelation coefficients, partial autocorrelation coefficients

In [ ]:
# below are a few function you may find useful

def trim_series(x, p_cut =0.025):
    """ 
    Discards p_cut of the smallest and the largets values from the series
    Returns:
    -------
        redcued series
    """
    N = len(x)
    N_start = int(p_cut*N)
    N_end = int((1-p_cut)*N)
    sorted_x = sorted(x)
    return sorted_x[N_start:N_end]

def autocorr(x, t=1):
    """calculates autocorrelation coefficient with lag t """
    return numpy.corrcoef(numpy.array([x[:-t], x[t:]]))[0,1]

def trimmed_kurtosis(x):
    """ calculate kurtosis for series without extreme values"""
    trimmed_x = trim_series(x)
    return kurtosis(trimmed_x)

def trimmed_skew(x):
    """ calculate skew for series without extreme values"""
    trimmed_x = trim_series(x)
    return skew(trimmed_x)

In [ ]:
# check different features using the function below
# see if the values are different for steady and anomalous series

def feature(x):
    
    #YOUR CODE
    
    res = autocorr(x,1)
    return res

plt.figure(dpi = 100)
steady_df['scaled_series'].apply(feature).hist(bins = 10, alpha = 0.5, label = 'steady')
anomaly_df['scaled_series'].apply(feature).hist(bins = 10, alpha = 0.5, label = 'anomaly')
plt.xlabel('feature value')
plt.ylabel('counts')
plt.legend()

For now, we generate a set of features based on the functions mentioned above for both the series and its first derivative.

In [ ]:
features_steady = generate_all_features(steady_df['signal'])
features_steady['target'] = 1
# remove all bad values
features_steady = features_steady[~features_steady.isin([numpy.nan,
                                                            numpy.inf,
                                                            -numpy.inf]).any(1)]


features_anomaly = generate_all_features(anomaly_df['signal'])
features_anomaly['target'] = -1
# remove all bad values
features_anomaly = features_anomaly[~features_anomaly.isin([numpy.nan,
                                                            numpy.inf,
                                                            -numpy.inf]).any(1)]


# combine the to data frames into one
features = pandas.concat([features_anomaly, features_steady])
features.shape

## Unsupervised Model: Elliptical Envelope

https://scikit-learn.org/stable/modules/generated/sklearn.covariance.EllipticEnvelope.html

### Data splitting for unsupervised learning: introducing only good examples during model training

We take 90% of the good data to use for training. The rest is used for testing. All of the anomalous data only appear during the testing.


In [ ]:
# reshaffle the data
features_steady = features_steady.sample(frac = 1, random_state = 25)

#define the split point
n_split = int(features_steady.shape[0]*0.9)
train_steady = features_steady[:n_split]
test_steady = features_steady[n_split:]

In [ ]:
# do the same for anomalous cases
# YOUR CODE



# ANSWER
# reshaffle the data
features_anomaly = features_anomaly.sample(frac = 1, random_state = 25)

#define the split point
n_split = int(features_anomaly.shape[0]*0.0)
train_anomaly = features_anomaly[:n_split]
test_anomaly = features_anomaly[n_split:]

In [ ]:
# combine the data and convert pandas.DataFrame object to a numpy.array

train = pandas.concat([train_steady, train_anomaly ]).reset_index(drop=True)
test = pandas.concat([test_steady, test_anomaly ]).reset_index(drop=True)

X_train = train.drop(columns =['target']).values
y_train = train['target'].values
X_test = test.drop(columns =['target']).values
y_test = test['target'].values


In [ ]:
from sklearn.covariance import EllipticEnvelope
from sklearn.decomposition import PCA

# reduce the number of variables to 2

scaler =  StandardScaler().fit(X_train) 
X_train_scaled = scaler.transform(X_train)
pca = PCA(2).fit(X_train_scaled)
train_pca = pca.transform(X_train_scaled)

X_test_scaled = scaler.transform(X_test)
test_pca = pca.transform(X_test_scaled)

In [ ]:
# Let us visualize the data using the two new coordinates
plt.figure(dpi = 200, figsize = (4,4))
plt.scatter(train_pca[:, 0], train_pca[:, 1], alpha = 0.4, label = 'training set',  edgecolor = 'none')
plt.scatter(test_pca[y_test == 1, 0], test_pca[y_test == 1, 1], alpha = 0.4,label = 'test set steady', edgecolor = 'none')
plt.scatter(test_pca[y_test == -1, 0], test_pca[y_test == -1, 1], alpha = 0.4,label = 'test set anomaly',edgecolor = 'none')
plt.xlabel('PCA1')
plt.ylabel('PCA2')
plt.xlim((-100, 75))
plt.ylim((-50, 100))
plt.legend()

In [ ]:
# combine dimensionality reduction and 
unsup_model = Pipeline([('scaler', StandardScaler()),
                 ('pca', PCA(2)),
                 ('model', EllipticEnvelope(assume_centered=False,
                                            random_state = 50,
                                            contamination = 0.0 # change if you think that some anomalies are in the 'steady' data
                                           ))])

unsup_model.fit(X_train) # notice that no target values (y_train) are needed 

### Evaluating model performance

Consider following metricses:
* accuracy
* confusion matrix (separates tru positive, true negative, false positive and false negative)
* recall (what portion of a single class is correctly identified)
* precision (how many prediction of a single class are correct)

More information at https://en.wikipedia.org/wiki/Confusion_matrix

In [ ]:
print(f'Accuracy is {numpy.round(accuracy_score(y_test, unsup_model.predict(X_test)),2)}')

In [ ]:
confusion_matrix(y_test, unsup_model.predict(X_test))

In [ ]:
print(f'Recall for steady is {numpy.round(recall_score(y_test, unsup_model.predict(X_test)),3)}')
print(f'Recall for anomaly is {numpy.round(recall_score(-y_test, -unsup_model.predict(X_test)),3)}')


# # YOUR CODE HERE
print(f'Precision for steady is {numpy.round(precision_score(y_test, ... ),3)}')
print(f'Precision for anomaly is {numpy.round(precision_score(-y_test, ... ),3)}')


# # ANSWER
# print(f'Precision for steady is {numpy.round(precision_score(y_test, unsup_model.predict(X_test)),3)}')
# print(f'Precision for anomaly is {numpy.round(precision_score(-y_test, -unsup_model.predict(X_test)),3)}')

In [ ]:
# visualize the envelope
from sklearn.covariance import EmpiricalCovariance

plt.figure(dpi = 200, figsize = (4,4))
emp_cov = EmpiricalCovariance().fit(pca.transform(X_train))
xx, yy = numpy.meshgrid(numpy.linspace(-100, 75, 100),
                     numpy.linspace(-50, 100, 100))
zz = numpy.c_[xx.ravel(), yy.ravel()]
mahal_emp_cov = emp_cov.mahalanobis(zz)
mahal_emp_cov = mahal_emp_cov.reshape(xx.shape)
emp_cov_contour = plt.contour(xx, yy, numpy.sqrt(mahal_emp_cov),
                              cmap=plt.cm.PuBu_r, linestyles='dashed', levels = 20)
plt.scatter(train_pca[:, 0], train_pca[:, 1], alpha = 0.4, label = 'training set',  edgecolor = 'none')
plt.scatter(test_pca[y_test == 1, 0], test_pca[y_test == 1, 1], alpha = 0.4,label = 'test set steady', edgecolor = 'none')
plt.scatter(test_pca[y_test == -1, 0], test_pca[y_test == -1, 1], alpha = 0.4,label = 'test set anomaly',edgecolor = 'none')
plt.xlim((-100, 75))
plt.ylim((-50, 100))
plt.xlabel('PCA1')
plt.ylabel('PCA2')
plt.legend()

## Supervised Model: Logistic Regression

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

### Data splitting for supervised learning: evenly split the data between the training and the test sets

We take 80% of the good and anomalous data to use for training

In [ ]:
# reshaffle the data
features_steady = features_steady.sample(frac = 1, random_state = 25)

#define the split point
n_split = int(features_steady.shape[0]*0.8)
train_steady = features_steady[:n_split]
test_steady = features_steady[n_split:]

# do the same for anomalous cases

# reshaffle the data
features_anomaly = features_anomaly.sample(frac = 1, random_state = 25)

#define the split point
n_split = int(features_anomaly.shape[0]*0.8)
train_anomaly = features_anomaly[:n_split]
test_anomaly = features_anomaly[n_split:]

In [ ]:
# combine the data and convert pandas.DataFrame object to a numpy.array

train = pandas.concat([train_steady, train_anomaly ]).reset_index(drop=True)
test = pandas.concat([test_steady, test_anomaly ]).reset_index(drop=True)

X_train = train.drop(columns =['target']).values
y_train = train['target'].values
X_test = test.drop(columns =['target']).values
y_test = test['target'].values

In [ ]:
from sklearn.linear_model import LogisticRegression

sup_model = Pipeline([('scaler', StandardScaler()),
                 ('logreg', LogisticRegression(class_weight= 'balanced', random_state=50, C = 100))]) 

sup_model.fit(X_train, y_train) # both values and targets are needed

### Evaluating model performance

Evaluate the model performance according to the same metricses as for the unsuprvised model

In [ ]:
print(f'Accuracy is {numpy.round(accuracy_score(y_test, sup_model.predict(X_test)),2)}')

In [ ]:
confusion_matrix(y_test, sup_model.predict(X_test))

In [ ]:
# calculate precision and recalls for both classes
# # YOUR CODE HERE


# # ANSWER
# print(f'Recall for steady is {numpy.round(recall_score(y_test, sup_model.predict(X_test)),3)}')
# print(f'Recall for anomaly is {numpy.round(recall_score(-y_test, -sup_model.predict(X_test)),3)}')

# print(f'Precision for steady is {numpy.round(precision_score(y_test, sup_model.predict(X_test)),3)}')
# print(f'Precision for anomaly is {numpy.round(precision_score(-y_test, -sup_model.predict(X_test)),3)}')

In [ ]:
# can predict probabilities for being steady data
sup_model.predict_proba(X_test[:5])[:, 1]

### Interpret the model's weights

Model weights can tell about the importance of the corresponding features. A large absolute value of a coefficient means that a small change of the variable leads to a large change of the probability

In [ ]:
coefficients = numpy.abs(sup_model['logreg'].coef_).flatten()
feature_names = numpy.array(train.drop(columns =['target']).columns)

feature_importance = pandas.Series(coefficients, index = feature_names).sort_values(ascending = True)
plt.figure(dpi = 150, figsize = (4, 5))
feature_importance.plot.barh()
plt.title('Feature Importance')

## Making prediction on new data

In [ ]:
# generate series
# YOUR CODE
# s = 


# EXAMPLE
s = numpy.ones(100) + numpy.random.random(100)

In [ ]:
from collections import namedtuple
measurement = namedtuple('time_series', 'uid signal')
new_series = pandas.DataFrame([measurement('new', s)])

# generate features
features_new_series = generate_all_features(new_series['signal'])

# predict with unsupervised model
unsup_pred = unsup_model.predict(features_new_series.values)
unsup_pred =('steady' if unsup_pred == 1 else 'anomaly')
print(f'Prediction of the unsupervised model is *{unsup_pred}*' )

# predict with supervised model
sup_pred = sup_model.predict(features_new_series.values)
sup_pred =('steady' if sup_pred == 1 else 'anomaly')
print(f'Prediction of the unsupervised model is *{sup_pred}*' )
